## KGWAS Disease critical network
This notebook showcases an example to generate (1) KGWAS network weights for each edge, measuring the importance of the edge to explain the disease GWAS signals and (2) variant interpretation graph where for each variant, it retrieves the top K nodes that are most important to explain the GWAS signals of the variant.

This code assumes that you have the model folder `./data/model/test` saved. Feel free to modify the path to your own model folder.

### Note
Please in the site package torch_geometric/nn/conv/hetero_conv.py, change the `group` function into this:

```python
def group(xs: List[Tensor], aggr: Optional[str]) -> Optional[Tensor]:
    if len(xs) == 0:
        return None
    elif aggr is None:
        return torch.stack(xs, dim=1)
    elif len(xs) == 1:
        return xs[0]
    elif isinstance(xs[0], tuple):
        return xs
    else:
        out = torch.stack(xs, dim=0)
        out = getattr(torch, aggr)(out, dim=0)
        out = out[0] if isinstance(out, tuple) else out
        return out
```


In [1]:
import sys
sys.path.append('../')

from kgwas import KGWAS, KGWAS_Data
data = KGWAS_Data(data_path = './data/')
data.load_kg()

data.load_external_gwas(example_file = True)
data.process_gwas_file()
data.prepare_split()

run = KGWAS(data, device = 'cuda:9', exp_name = 'test')
run.load_pretrained('./data/model/test')
df_network_weight, df_variant_interpretation, disease_critical_network = run.get_disease_critical_network(variant_threshold = 5e-8, 
                                                                                                            magma_path = None, magma_threshold = 0.05, program_threshold = 0.05,
                                                                                                            K_neighbors = 3, num_cpus = 1)

All required data files are present.
--loading KG---
--using enformer SNP embedding--
--using random go embedding--
--using ESM gene embedding--
Loading example GWAS file...
Example file already exists locally.
Loading GWAS file from ./data/biochemistry_Creatinine_fastgwa_full_10000_1.fastGWA...
Using ldsc weight...
ldsc_weight mean:  0.9999999999999993
Retrieving weights...
Aggregating across node types...
Start generating disease critical network...
No filters... Using all genes and gene programs...


/dfs/user/kexinh/miniconda3/envs/a100_env/lib/python3.8/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
/dfs/user/kexinh/miniconda3/envs/a100_env/lib/python3.8/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


Disease critical network finished generating...
Generating variant interpretation networks...
Number of hit snps:  3


100%|██████████| 3/3 [00:22<00:00,  7.50s/it]


In [2]:
df_network_weight

,h_idx,t_idx,weight,h_type,rel_type,t_type,layer
0,11105.0,3.0,-0.005648,Gene,rev_ABC,SNP,l1
1,11105.0,4.0,-0.006695,Gene,rev_ABC,SNP,l1
2,10667.0,8.0,-0.007284,Gene,rev_ABC,SNP,l1
3,11105.0,8.0,-0.007112,Gene,rev_ABC,SNP,l1
4,10667.0,11.0,-0.007329,Gene,rev_ABC,SNP,l1
...,...,...,...,...,...,...,...
10,1191.0,1147.0,0.004166,CellularComponent,rev_Gene-NotColocalizes-CellularComponent,Gene,l2
0,2423.0,19025.0,0.439045,MolecularFunction,rev_Gene-NotContributes-MolecularFunction,Gene,l2
1,768.0,12181.0,0.203226,MolecularFunction,rev_Gene-NotContributes-MolecularFunction,Gene,l2
2,4511.0,12992.0,0.127140,MolecularFunction,rev_Gene-NotContributes-MolecularFunction,Gene,l2


In [3]:
df_variant_interpretation

,h_idx,t_idx,importance,h_type,t_type,rel_type,h_id,t_id,QUERY_SNP
393735,4238.0,115005.0,0.707107,Gene,SNP,Exon,CPS1,rs1047891,rs1047891
1098740,17670.0,4238.0,1.779486,Gene,Gene,PhysicalAssociation,RPL35,CPS1,rs1047891
573324,8800.0,4238.0,0.425998,Gene,Gene,PhysicalAssociation,HSPA5,CPS1,rs1047891
786046,13187.0,4238.0,0.019682,Gene,Gene,DosageLethality,MYC,CPS1,rs1047891
28855,13165.0,4238.0,-0.035167,BiologicalProcess,Gene,Associates,Nitric oxide metabolic process,CPS1,rs1047891
30283,14457.0,4238.0,-0.035197,BiologicalProcess,Gene,Associates,Homocysteine metabolic process,CPS1,rs1047891
14116,6294.0,4238.0,-0.035242,BiologicalProcess,Gene,Associates,Triglyceride catabolic process,CPS1,rs1047891
280494,114935.0,4238.0,1.473419,SNP,Gene,PCHi-C,rs116492934,CPS1,rs1047891
280526,114958.0,4238.0,1.433980,SNP,Gene,PCHi-C,rs148584272,CPS1,rs1047891
280523,114956.0,4238.0,1.381602,SNP,Gene,PCHi-C,rs138424013,CPS1,rs1047891


In default, it does not use any filtering on the genes and programs. But we highly suggest you do so! You can do it by simply feeding the MAGMA result file to `magma_path`. For how to run MAGMA, check out [this notebook](run_magma.ipynb)!